> **THIS KERNEL WILL RESULT IN PUBLIC LB AROUND 0.78, BUT IF YOU PLAY AROUND WITH TRANSOFORMATIONS, IMAGE_SIZE, TRAINING CYCLES IT CAN RESULT TO >0.795**

This is just a starter kernel, I provide all the basic stuff, you have to figure out how you can utilize it to achieve higher score 
NOTE:
    - This kernel doesnt use optimized KAPPA
    - I have trained locally on dataset that was hybrid of new and old competiions (balanced classes), and just uploaded weights
    - You dont need to do anything fancy to achieve top scores all the hints are in discussions and kernels
    - I have taken code from public kernels, please let me know if I fogot to mentioned 

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
%matplotlib inline  
from sklearn.model_selection import StratifiedKFold
from joblib import load, dump
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import confusion_matrix
from fastai import *
from fastai.vision import *
from fastai.callbacks import *
from torchvision import models as md
from torch import nn
from torch.nn import functional as F
import re
import math
import collections
from functools import partial
from torch.utils import model_zoo
from sklearn import metrics
from collections import Counter
import json

from efficientnet_pytorch import EfficientNet


In [2]:
#making model
md_ef = EfficientNet.from_pretrained('efficientnet-b5', num_classes=1)

Downloading: "http://storage.googleapis.com/public-models/efficientnet-b5-586e6cc6.pth" to /home/iwai/.torch/models/efficientnet-b5-586e6cc6.pth
100%|██████████| 122398170/122398170 [00:05<00:00, 23250156.81it/s]


AttributeError: 'NoneType' object has no attribute 'missing_keys'

In [ ]:
def get_df():
    base_image_dir = os.path.join('..', 'preprocessed/')
    train_dir = os.path.join(base_image_dir,'train_crop/')
    df = pd.read_csv('../input/train.csv')
    df['path'] = df['id_code'].map(lambda x: os.path.join(train_dir,'{}.npy'.format(x)))
    df = df.drop(columns=['id_code'])
    df = df.sample(frac=1).reset_index(drop=True) #shuffle dataframe
    test_df = pd.read_csv('../input/sample_submission.csv')
    return df, test_df

df, test_df = get_df()

In [ ]:
#you can play around with tfms and image sizes
bs = 64
sz = 224
tfms = get_transforms(do_flip=True,flip_vert=True)

In [3]:
data = (ImageList.from_df(df=df,path='./',cols='path') 
        .split_by_rand_pct(0.2) 
        .label_from_df(cols='diagnosis',label_cls=FloatList) 
        .transform(tfms,size=sz,resize_method=ResizeMethod.SQUISH,padding_mode='zeros') 
        .databunch(bs=bs,num_workers=4) 
        .normalize(imagenet_stats)  
       )

NameError: name 'df' is not defined

In [ ]:
def qk(y_pred, y):
    return torch.tensor(cohen_kappa_score(torch.round(y_pred), y, weights='quadratic'), device='cuda:0')

In [ ]:
learn = Learner(data, 
                md_ef, 
                metrics = [qk], 
                model_dir="models").to_fp16()

learn.data.add_test(ImageList.from_df(test_df,
                                      '../preprocessed',
                                      folder='test_crop',
                                      suffix='.npy'))



Try to fit model here:
> learn.fit_one_cycle(10, 1e-3)

In [ ]:
learn.fit_one_cycle(50, 1e-3)

In [ ]:
# learn.load('abcdef');

In [ ]:
#https://www.kaggle.com/abhishek/optimizer-for-quadratic-weighted-kappa
class OptimizedRounder(object):
    def __init__(self):
        self.coef_ = 0

    def _kappa_loss(self, coef, X, y):
        X_p = np.copy(X)
        for i, pred in enumerate(X_p):
            if pred < coef[0]:
                X_p[i] = 0
            elif pred >= coef[0] and pred < coef[1]:
                X_p[i] = 1
            elif pred >= coef[1] and pred < coef[2]:
                X_p[i] = 2
            elif pred >= coef[2] and pred < coef[3]:
                X_p[i] = 3
            else:
                X_p[i] = 4

        ll = metrics.cohen_kappa_score(y, X_p, weights='quadratic')
        return -ll

    def fit(self, X, y):
        loss_partial = partial(self._kappa_loss, X=X, y=y)
        initial_coef = [0.5, 1.5, 2.5, 3.5]
        self.coef_ = sp.optimize.minimize(loss_partial, initial_coef, method='nelder-mead')
        print(-loss_partial(self.coef_['x']))

    def predict(self, X, coef):
        X_p = np.copy(X)
        for i, pred in enumerate(X_p):
            if pred < coef[0]:
                X_p[i] = 0
            elif pred >= coef[0] and pred < coef[1]:
                X_p[i] = 1
            elif pred >= coef[1] and pred < coef[2]:
                X_p[i] = 2
            elif pred >= coef[2] and pred < coef[3]:
                X_p[i] = 3
            else:
                X_p[i] = 4
        return X_p

    def coefficients(self):
        return self.coef_['x']

In [ ]:
def run_subm(learn=learn, coefficients=[0.5, 1.5, 2.5, 3.5]):
    opt = OptimizedRounder()
    preds,y = learn.get_preds(DatasetType.Test)
    tst_pred = opt.predict(preds, coefficients)
    test_df.diagnosis = tst_pred.astype(int)
    test_df.to_csv('submission.csv',index=False)
    print ('done')

In [ ]:
run_subm()